In [1]:
import os
os.chdir("../")
%pwd

'd:\\Cite-What-You-Type'

In [2]:
import base64
from io import BytesIO
from PIL import Image

In [3]:
file_path="pdfs/Documentation-Project.pdf"

In [57]:
from pinecone import Pinecone
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from pinecone_text.sparse import BM25Encoder
from langchain_community.retrievers import PineconeHybridSearchRetriever
from Multi_Modal.SeperationAndSummarization import summarize_chunks
from langchain_community.document_loaders import UnstructuredWordDocumentLoader,UnstructuredExcelLoader,UnstructuredPowerPointLoader
from unstructured.partition.docx import partition_docx
from unstructured.partition.pptx import partition_pptx
from unstructured.partition.pdf import partition_pdf
from  unstructured.chunking.title import chunk_by_title
from IPython.display import HTML

In [20]:
load_dotenv()

True

In [6]:
def extract_pdf_elements(filepath: str) -> list:
    elements = partition_pdf(
        filename=filepath,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=['Image'],
        extract_image_block_to_payload=True,
    )
    print("Total elements extracted -> ",len(elements))
    return elements


In [7]:
def create_chunks_by_title(elements):
    chunks = chunk_by_title(
        elements,
        max_characters=2000,
        combine_text_under_n_chars=500
    )

    print("Chunks created: ", len(chunks))
    return chunks

In [8]:
def get_chunks(filepath:str):
    elements = extract_pdf_elements(filepath)
    chunk = create_chunks_by_title(elements)

    return chunk

In [9]:
chunk = get_chunks(file_path)

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Total elements extracted ->  141
Chunks created:  14


In [14]:
summarized_chunk = summarize_chunks(chunk)

...Processing Chunk ...
Processed Chunk 1/14
Types Found:  ['Table', 'Image', 'text']
Tables: 1 , Image: 1
Creating Summary...
Successfully Summarized
Preview:Here is the searchable description:

### 1. Key Facts, Exact Numbers, and Data Points
*   **Project ...
Processed Chunk 2/14
Types Found:  ['text']
Tables: 0 , Image: 0
No tables or Image Found
Processed Chunk 3/14
Types Found:  ['text']
Tables: 0 , Image: 0
No tables or Image Found
Processed Chunk 4/14
Types Found:  ['Table', 'text']
Tables: 1 , Image: 0
Creating Summary...
Successfully Summarized
Preview:**SEARCHABLE DESCRIPTION:**

**1. Key facts, exact numbers, and data points (from text and tables)**...
Processed Chunk 5/14
Types Found:  ['Table', 'text']
Tables: 1 , Image: 0
Creating Summary...
Successfully Summarized
Preview:**SEARCHABLE DESCRIPTION:**

**1. Key facts, exact numbers, and data points (from text and tables):*...
Processed Chunk 6/14
Types Found:  ['Table', 'text']
Tables: 2 , Image: 0
Creating Summary...
Suc

In [10]:
text1=chunk[10].to_dict()
text1

{'type': 'CompositeElement',
 'element_id': 'c9002b91-d5af-4764-bbb2-cc378fd1c73d',
 'text': 'Performance Metrics:\n\nConfusion_Matrix:\n\nConfusion Matrix Predicted\n\nClassification_report:\n\nClassification_report:\n\nModel Deployment:\n\nThe final phase of the project focused on operationalizing the high-performance DeBERTa classification model into a scalable, accessible, end-to-end web application. This was achieved using a modern, containerized architecture deployed on Google Cloud Platform (GCP).\n\nBackend: Prediction API (Model Serving)\n\nThe core function of generating predictions was exposed via a high-speed API built on FastAPI.\n\n● API Framework (FastAPI): FastAPI was chosen for its modern, asynchronous capabilities and high performance, making it ideal for serving the computationally intensive Transformer model with low latency.\n\n● Containerization (Docker): The DeBERTA model, all Python\n\ndependencies (e.g., PyTorch, Hugging Face transformers), and the FastAPI serv

In [38]:
for elem in chunk[10].metadata.orig_elements:
    print(type(elem).__name__)


Title
Title
Image
Image
Text
Title
NarrativeText
NarrativeText
NarrativeText
ListItem
Text
ListItem
Text
ListItem


In [40]:
def seperate_content_types(chunk):

    content_data ={
        "text": chunk.text,
        "tables":[],
        "images":[],
        "types":['text']
    }

    if hasattr(chunk ,"metadata") and hasattr(chunk.metadata, "orig_elements"):
        for element in chunk.metadata.orig_elements:

            element_type = type(element).__name__

            if element_type =="Table":
                content_data['types'].append('Table')
                table_html = getattr(element.metadata , 'text_as_html', element.text)
                content_data['tables'].append(table_html)

            elif element_type == "Image":
                if hasattr(element,"metadata") and hasattr(element.metadata ,"image_base64"):
                    content_data['types'].append("Image")
                    content_data['images'].append(element.metadata.image_base64)
    
    content_data['types'] = list(set(content_data['types']))
    return content_data

In [47]:
sep=seperate_content_types(chunk[10])
sep

{'text': 'Performance Metrics:\n\nConfusion_Matrix:\n\nConfusion Matrix Predicted\n\nClassification_report:\n\nClassification_report:\n\nModel Deployment:\n\nThe final phase of the project focused on operationalizing the high-performance DeBERTa classification model into a scalable, accessible, end-to-end web application. This was achieved using a modern, containerized architecture deployed on Google Cloud Platform (GCP).\n\nBackend: Prediction API (Model Serving)\n\nThe core function of generating predictions was exposed via a high-speed API built on FastAPI.\n\n● API Framework (FastAPI): FastAPI was chosen for its modern, asynchronous capabilities and high performance, making it ideal for serving the computationally intensive Transformer model with low latency.\n\n● Containerization (Docker): The DeBERTA model, all Python\n\ndependencies (e.g., PyTorch, Hugging Face transformers), and the FastAPI server were encapsulated into a single Docker Container : This ensured the environment w

In [50]:
from transformers import BlipProcessor, BlipForConditionalGeneration
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
model.eval()

In [56]:
def base_64_to_pil_image(image):
    img_bytes = base64.b64decode(image)
    img_buffer = BytesIO(img_bytes)
    img =Image.open(img_buffer).convert("RGB")
    return img


In [57]:
def caption_from_element(element:list) ->list[str]:
    text=[]
    for image in element:
        pil_img = base_64_to_pil_image(image)
        inputs=processor(images=pil_img, return_tensors="pt")
        out = model.generate(**inputs, max_new_tokens=64)

        text.append(processor.decode(out[0],skip_sepecial_token=True))
    return text


In [58]:
caption_from_element(sep["images"])

['a graph with the number of the two squares [SEP]',
 'a screenshot of a computer screen with a text that reads, ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` ` `']

In [15]:
summarized_chunk

[Document(metadata={'original_content': '{"raw_text": "Project Name: Fake News Detector\\n\\ncs2011: INtroDuctIoN to aI aND mL INstructor: Dr. sobhaN kaNtI Dhara\\n\\nDePartmeNt oF comPuter scIeNce aND eNgINeerINg\\n\\n(cse)\\n\\nNIt rourkeLa\\n\\nDate oF submIssIoN: - 19/11/2025\\n\\nGroup Members:\\n\\nDesign And Development Data Collection and Database Management Documentation and Presentation Team 1. Raghav Iyer Ramaswamy (124AI0004) 2. Soumya Ranjan Bhoi (124AI0010) 3. Soumyadeep Sahoo (124AI0012) 1.Saswat Sahu (124AI0017) 2. Rahul Padhy (124AI0006) 3. Shakti Prasad Sahoo (124AI0019) 1.Hriday Gupta (124AI0007) 2.Srikant Panigrahy (124AI0023) 3.Unays Sahir Alam (124AI0003)\\n\\nTeam", "table_html": ["<table><thead><tr><th>Design And Development Team</th><th>1. Raghav lyer Ramaswamy (124AI0004) 2. Soumya Ranjan Bhoi (124AI0010) 3. Soumyadeep Sahoo (124AI0012)</th></tr></thead><tbody><tr><td>Data Collection and Database</td><td>1.Saswat Sahu (124A10017)</td></tr><tr><td></td><td>3. S

In [16]:
bm25 = BM25Encoder.default()

In [30]:
pc=Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

In [31]:
pc.list_indexes().names()

['hybrid-search', 'soumya-index']

In [97]:
index_name = "hybrid-demo"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="dotproduct", 
        spec={"serverless": {"cloud": "gcp", "region": "us-central1"}}
    )

In [42]:
hasattr(summarized_chunk[0],"page_content")

True

In [41]:
summarized_chunk[0].page_content

'Here is the searchable description:\n\n### 1. Key Facts, Exact Numbers, and Data Points\n*   **Project Name:** Fake News Detector\n*   **Institution:** National Institute of Technology Rourkela (NIT Rourkela)\n*   **Department:** Department of Computer Science and Engineering (CSE)\n*   **Course Code:** cs2011\n*   **Course Name:** Introduction to AI and ML (Introduction to Artificial Intelligence and Machine Learning)\n*   **Instructor:** Dr. Sobhan Kanti Dhara\n*   **Date of Submission:** 19/11/2025\n*   **Total Group Members:** 9 students organized into 3 teams.\n\n*   **Group Members and Teams:**\n    *   **Design And Development Team:**\n        *   Raghav Iyer Ramaswamy (Student ID: 124AI0004)\n        *   Soumya Ranjan Bhoi (Student ID: 124AI0010)\n        *   Soumyadeep Sahoo (Student ID: 124AI0012)\n    *   **Data Collection and Database Management Team:**\n        *   Saswat Sahu (Student ID: 124AI0017)\n        *   Rahul Padhy (Student ID: 124AI0006)\n        *   Shakti Pra

In [48]:
def get_text(document) ->list:
    text=[]
    for doc in document:
        if hasattr(doc,"page_content"):
            text.append(doc.page_content)
    return text
            

In [49]:
text_list = get_text(summarized_chunk)

In [98]:
index=pc.Index(index_name)

In [96]:
emb_model=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [56]:
bm25.fit(text_list)

100%|██████████| 14/14 [00:00<00:00, 320.28it/s]


In [59]:
bm25.dump("bm25_enc.json")

In [62]:
retreiver = PineconeHybridSearchRetriever(embeddings=emb_model,sparse_encoder=bm25,index=index)

In [72]:
values=bm25.encode_documents(text_list[0])['values']

In [78]:
len(values)

161

In [94]:
len(emb_model.embed_query(text_list[0]))

3072

In [99]:
batch_size = 2
vectors_to_upsert = []

for i, text in enumerate(text_list):
    vector = {
        "id": f"doc{i}",
        "values": emb_model.embed_query(text),
        "sparse_values": bm25.encode_documents(text),
        "metadata": {"text": text}
    }
    vectors_to_upsert.append(vector)
    
    if len(vectors_to_upsert) >= batch_size:
        index.upsert(vectors_to_upsert)
        vectors_to_upsert = []

if vectors_to_upsert:
    index.upsert(vectors_to_upsert)


In [105]:
retreiver = PineconeHybridSearchRetriever(embeddings=emb_model,sparse_encoder=bm25,index=index,text_key="text" )

In [106]:
retreiver.invoke("member invoved in the project")

[Document(metadata={'score': 0.537451625}, page_content='Here is the searchable description:\n\n### 1. Key Facts, Exact Numbers, and Data Points\n*   **Project Name:** Fake News Detector\n*   **Institution:** National Institute of Technology Rourkela (NIT Rourkela)\n*   **Department:** Department of Computer Science and Engineering (CSE)\n*   **Course Code:** cs2011\n*   **Course Name:** Introduction to AI and ML (Introduction to Artificial Intelligence and Machine Learning)\n*   **Instructor:** Dr. Sobhan Kanti Dhara\n*   **Date of Submission:** 19/11/2025\n*   **Total Group Members:** 9 students organized into 3 teams.\n\n*   **Group Members and Teams:**\n    *   **Design And Development Team:**\n        *   Raghav Iyer Ramaswamy (Student ID: 124AI0004)\n        *   Soumya Ranjan Bhoi (Student ID: 124AI0010)\n        *   Soumyadeep Sahoo (Student ID: 124AI0012)\n    *   **Data Collection and Database Management Team:**\n        *   Saswat Sahu (Student ID: 124AI0017)\n        *   Rah